# Comparing DeepSeek-R1-Distill-Qwen-1.5B with TinyLlama for Reasoning Tasks

In this notebook, I compared two compact language models: **DeepSeek R1 Distill Qwen 1.5B** and **TinyLlama**. Both models can run on CPUs. DeepSeek R1 Distill Qwen 1.5B, a recent release, emphasizes enhanced reasoning capabilities achieved through reinforcement learning techniques. TinyLlama, an open-source model, offers a balance between performance and resource efficiency.

To evaluate their reasoning abilities, I present each model with three distinct tasks, each targeting a different aspect of reasoning. After obtaining their responses, I perform a comparative analysis to assess their strengths and limitations in handling these tasks. This evaluation aims to provide insights into the practical applications and effectiveness of these models in reasoning scenarios. 

### Python Requirements for this notebook
I ran this code on `Python 3.12.3`. The list of packages needed is here [requirements.txt](../requirements.txt).


### Imports

In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from rich.console import Console
from rich.table import Table
from rich.markdown import Markdown

### Sample Reasoning Prompts Used with Both Models

In [20]:
# # Define the prompts
prompts = [
    "A farmer has chickens and cows on his farm. There are a total of 30 heads and 100 legs. How many chickens and how many cows are there?",
    "Observe the following sequence of numbers: 2, 4, 8, 16. Predict the next number in the sequence and explain the pattern.",
    "Hand is to glove as foot is to ____?"
]

### Loading `TinyLlama` and `DeepSeek R1 Distill Qwen 1.5B` models for local inferencing

In [21]:
# Load the TinyLlama model and tokenizer
tinyllama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tinyllama_model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Load the DeepSeek R1 Distill Qwen 1.5B model and tokenizer
deepseek_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
deepseek_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

### Prompting both Models with Reasoning Questions and Printing Respones

In [22]:
from rich.console import Console
from rich.markdown import Markdown

# Function to generate a response from a model
def generate_response(model, tokenizer, prompt):
    # Enhance the prompt with summarization instructions
    instruction = "Please provide a complete and concise summary of the following within 512 tokens:\n"
    enhanced_prompt = instruction + prompt

    # Tokenize the enhanced prompt
    inputs = tokenizer(enhanced_prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,  # Limit the generated response to 512 tokens
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    # Extract the generated tokens, excluding the input prompt
    generated_tokens = outputs[:, inputs['input_ids'].shape[-1]:]
    response = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return response

# Create a list to store the results
results = []

# Iterate over the prompts and get responses from both models
for prompt in prompts:
    tinyllama_response = generate_response(tinyllama_model, tinyllama_tokenizer, prompt)
    deepseek_response = generate_response(deepseek_model, deepseek_tokenizer, prompt)
    results.append({
        "Prompt": prompt,
        "TinyLlama": tinyllama_response,
        "DeepSeek R1 Distill Qwen 1.5B": deepseek_response
    })

# Initialize the console for rich output
console = Console()

# Display each prompt and its corresponding responses
for result in results:
    markdown_content = f"""
**Prompt:**

{result['Prompt']}

**TinyLlama:**

{result['TinyLlama']}

**DeepSeek R1 Distill Qwen 1.5B:**

{result['DeepSeek R1 Distill Qwen 1.5B']}
"""
    console.print(Markdown(markdown_content))


Prompt:                                                                                                            

A farmer has chickens and cows on his farm. There are a total of 30 heads and 100 legs. How many chickens and how  
many cows are there?                                                                                               

TinyLlama:                                                                                                         

DeepSeek R1 Distill Qwen 1.5B:                                                                                     

(Please answer in a clear and concise manner.) To solve this problem, I need to set up a system of equations. Let  
me denote the number of chickens as C and the number of cows as K.                                                 

Since each chicken and each cow has one head, the total number of heads is the sum of chickens and cows. So, C + K 
= 30.                                                                                                              

Each chicken has two legs and each cow has four legs, so the total number of legs is 2C + 4K = 100.                

Now, I can solve this system of equations. Let me subtract twice the first equation from the second equation to    
eliminate the variable for chickens.                                                                               

2C + 4K = 100                                                                                                      

2(C + K) = 60                                                                                                      

Subtracting these two equations: (2C + 4K) - 2(C + K) = 100 - 60                                                   

Which simplifies to 2C + 4K - 2C - 2K = 40                                                                         

So, 2K = 40                                                                                                        

Dividing both sides by 2: K = 20                                                                                   

Now that I know there are 20 cows, I can substitute this back into the first equation to find the number of        
chickens.                                                                                                          

C + 20 = 30                                                                                                        

Subtracting 20 from both sides: C = 10                                                                             


Therefore, there are 10 chickens and 20 cows on the farm.                                                          

The farmer has a total of 30 heads and 100 legs consisting of chickens and cows. To find the number of each, set up
the following system of equations:                                                                                 

 1 ( C + K = 30 ) (total heads)                                                                                    
 2 ( 2C + 4K = 100 ) (total legs)                                                                                  

Solving this system:                                                                                               

 • Subtract twice the first equation from the second: [ 2C + 4K - 2(C + K) = 100 - 60 \implies 2K = 40 \implies K =
   20 ]                                                                                                            
 • Substitute ( K = 20 ) into ( C + K = 30 ): [ C + 20 = 30 \implies C = 10 ]                                      

Answer: There are 10 chickens and 20 cows on the farm.

Prompt:                                                                                                            

Observe the following sequence of numbers: 2, 4, 8, 16. Predict the next number in the sequence and explain the    
pattern.                                                                                                           

TinyLlama:                                                                                                         

Solution: The sequence of numbers we have is 2, 4, 8, 16, which has a repeating pattern of 4, 8, 16, 2.            

Prediction: The next number in the sequence will be 10, which follows the pattern of 8, 16, 2.                     

Conclusion: The sequence of numbers we have is a repeating pattern of 4, 8, 16, 2, which follows the pattern of 8, 
16, 2. The next number in the sequence will be 10.                                                                 

DeepSeek R1 Distill Qwen 1.5B:                                                                                     

You must provide a complete and concise summary of the following within 512 tokens. Okay, so I need to figure out  
the next number in the sequence: 2, 4, 8, 16. Let me think about the pattern here. Each number seems to be double  
the previous one. So, 2 times 2 is 4, 4 times 2 is 8, and 8 times 2 is 16. That makes sense. If I continue this    
pattern, the next number should be 16 multiplied by 2, which is 32.                                                

Wait, but is there another way to look at this? Maybe it's powers of 2. 2 is 2 to the power of 1, 4 is 2 squared, 8
is 2 cubed, and 16 is 2 to the fourth power. So, the next number would be 2 to the fifth power, which is 32. That  
confirms my earlier thought.                                                                                       

I guess the key here is recognizing that each term is multiplied by 2 to get the next one. There's no addition or  
subtraction involved, just multiplication by 2. So, the pattern is consistent and straightforward.                 

Is there anything else I should consider? Maybe whether the sequence is geometric or arithmetic. Since each term is
multiplied by a constant factor (which is 2 in this case), it's a geometric sequence. The common ratio is 2, so the
next term should be 32.                                                                                            

I don't think there's a different pattern here. It seems pretty clear that each number is double the one before it.
So, confidently, the next number should be 32.                                                                     

Just to double-check, let's go through the sequence again: 2, 4, 8, 16, 32. Each time, 2 is multiplied by the      
previous number. Yep, that's consistent. No anomalies or unexpected numbers here.                                  

Therefore, the next number in the sequence is 32, following the pattern of multiplying by 2 each time.             

I think that's a solid conclusion. The pattern is consistent, and there's no need for any additional explanations  
beyond confirming the pattern.                                                                                     

Summary: The sequence 2, 4, 8, 16 follows a pattern where each number is double the previous one. The next number  
in the sequence is 32, as this follows the consistent multiplication by

Prompt:                                                                                                            

Hand is to glove as foot is to ____?                                                                               

TinyLlama:                                                                                                         

Please provide a complete and concise summary of the following within 512 tokens: The book ____ for the ____ year, 
and I'm sure you'll find it interesting. Please provide a complete and concise summary of the following within 512 
tokens: He's not a ____ person, but he's a ____ person. Please provide a complete and concise summary of the       
following within 512 tokens: She's a ____ person, but she's a ____ person. Please provide a complete and concise   
summary of the following within 512 tokens: He's a ____ person, but he's a ____ person. Please provide a complete  
and concise summary of the following within 512 tokens: He's a ____ person, and he's a ____ person. Please provide 
a complete and concise summary of the following within 512 tokens: He's a ____ person, and he's a ____ person.     
Please provide a complete and concise summary of the following within 512 tokens: He's a ____ person, and he's a   
____ person.                                                                                                       

DeepSeek R1 Distill Qwen 1.5B:                                                                                     

( ) To the right of the right side of the right side of the right side of the right side of the right side of the  
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right side of the right side of the right side of the right side of the right side of the right side of
the right side of the right side of the right side of the right side of the right side of the right side of the    
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right side of the right side of the right side of the right side of the right side of the right side of
the right side of the right side of the right side of the right side of the right side of the right side of the    
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right side of the right side of the right side of the right side of the right side of the right side of
the right side of the right side of the right side of the right side of the right side of the right side of the    
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right side of the right side of the right side of the right side of the right side of the right side of
the right side of the right side of the right side of the right side of the right side of the right side of the    
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right side of the right side of the right side of the right side of the right side of the right side of
the right side of the right side of the right side of the right side of the right side of the right side of the    
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right side of the right side of the right side of the right side of the right side of the right side of
the right side of the right side of the right side of the right side of the right side of the right side of the    
right side of the right side of the right side of the right side of the right side of the right side of the right  
side of the right